# **Create Hugging Face dataset for summarization task**

### **Load libraries**

In [33]:
%pip install -qq polars<1.22,>=1.20
%pip install -qq -U huggingface_hub
%pip install -qq -U datasets

/bin/bash: line 1: 1.22,: No such file or directory
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [3]:
%pip show polars
%pip show huggingface_hub
%pip show datasets

Name: polars
Version: 1.21.0
Summary: Blazingly fast DataFrame library
Home-page: https://www.pola.rs/
Author: 
Author-email: Ritchie Vink <ritchie46@gmail.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: cudf-polars-cu12
Note: you may need to restart the kernel to use updated packages.
Name: huggingface-hub
Version: 0.34.4
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, hf-xet, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, datasets, diffusers, gradio, gradio_client, peft, sentence-transformers, timm, tokenizers, torchtune, transformers
Note: you may need to restart the kernel to use updated packages.
Name: datasets
Version: 4.0.0
Summary: Huggin

### **Imports**

In [4]:
import zipfile
import os, sys
import shutil
import random
import subprocess
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa

import pyarrow.parquet as pq

from kaggle_secrets import UserSecretsClient

from datasets import Dataset, DatasetDict

from huggingface_hub import (
    Repository, 
    get_full_repo_name,
    login,
    upload_folder,
    hf_hub_download,
    HfApi
)


### **Hugging Face Login**

In [5]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
user_email = user_secrets.get_secret("user_email")
user_name = user_secrets.get_secret("user_name")

login(token=hf_token)

### **You will need to set up git, edit your email and name.**

In [6]:
def set_git_config(email, name):
    try:
        # Setting global user.email
        subprocess.run(["git", "config", "--global", "user.email", email], check=True)
        #print(f"Git user.email set to: {email}")
        
        # Setting the global user.name
        subprocess.run(["git", "config", "--global", "user.name", name], check=True)
        #print(f"Git user.name set to: {name}")
        
        # Check settings (optional)
        email_output = subprocess.run(["git", "config", "--global", "user.email"], capture_output=True, text=True, check=True)
        name_output = subprocess.run(["git", "config", "--global", "user.name"], capture_output=True, text=True, check=True)
        #print(f"Check - Email: {email_output.stdout.strip()}")
        #print(f"Check - Name: {name_output.stdout.strip()}")
        
    except subprocess.CalledProcessError as e:
        print(f"Error while setting up Git configuration: {e}")

In [7]:
set_git_config(user_email, user_name)

### **Load dataset to the polars**

In [8]:
path_input_file = os.path.join('/kaggle','input','text-for-summarize-nlpllm-task','summary_dataset_en.json')
polars_dataset = pl.read_json(path_input_file)

In [9]:
polars_dataset.head(3)

id,origin_text,summary_text,lenght_origin_text,lenght_summary_text
i64,str,str,i64,i64
0,"""By . Daily Mail Reporter . PUB…","""Cross-border violence began Fr…",3490,124
1,"""A man is suing after allegedly…","""Andrew Walls, 32, claims the u…",1934,309
2,"""The driver who was at the cont…","""William Rockefeller, 46, told …",6646,429


In [10]:
print('Number of rows:', polars_dataset.shape[0])
print('Number of columns:', polars_dataset.shape[1])

Number of rows: 317407
Number of columns: 5


In [11]:
print("lenght_origin_text:")
polars_dataset.select([
    pl.col("lenght_origin_text").min().alias("min"),
    pl.col("lenght_origin_text").mean().alias("mean"),
    pl.col("lenght_origin_text").median().alias("median"),
    pl.col("lenght_origin_text").max().alias("max"),
    pl.col("lenght_origin_text").std().alias("std_dev"),
])

lenght_origin_text:


min,mean,median,max,std_dev
i64,f64,f64,i64,f64
48,3865.262269,3513.0,296786,2194.774563


In [12]:
print("lenght_summary_text:")
polars_dataset.select([
    pl.col("lenght_summary_text").min().alias("min"),
    pl.col("lenght_summary_text").mean().alias("mean"),
    pl.col("lenght_summary_text").median().alias("median"),
    pl.col("lenght_summary_text").max().alias("max"),
    pl.col("lenght_summary_text").std().alias("std_dev"),
])

lenght_summary_text:


min,mean,median,max,std_dev
i64,f64,f64,i64,f64
14,329.896017,286.0,12344,208.44977


In [13]:
len(polars_dataset)

317407

### **Shuffle the dataset**

In [14]:
shuffled_polars_dataset = polars_dataset.sample(fraction=1.0, shuffle=True)

In [15]:
shuffled_polars_dataset.head(3)

id,origin_text,summary_text,lenght_origin_text,lenght_summary_text
i64,str,str,i64,i64
136532,"""By . Daily Mail Reporter . PUB…","""Tachowa Covington lived in an …",5441,256
250695,"""(CNN) -- The W Austin hotel is…","""Falling-glass incident is the …",1929,262
94770,"""Ancient human remains of what …","""Archaeologists unearthed the l…",5287,418


In [16]:
print('Number of rows:', polars_dataset.shape[0])
print('Number of columns:', polars_dataset.shape[1])

Number of rows: 317407
Number of columns: 5


## **Splitting the dataset into train (80%), validation (10%) and test (10%)**

In [17]:
np.random.seed(42)
total_rows = shuffled_polars_dataset.height
train_size = int(0.80 * total_rows)
train_size

253925

In [18]:
remaining_size = total_rows - train_size
val_size = int(0.50 * remaining_size)
val_size

31741

In [19]:
test_size = remaining_size - val_size
test_size

31741

In [20]:
train_df = shuffled_polars_dataset.sample(n=train_size, shuffle=True)
remaining_df = shuffled_polars_dataset.filter(~pl.col("id").is_in(train_df["id"]))
val_df = remaining_df.sample(n=val_size, shuffle=True)
test_df = remaining_df.filter(~pl.col("id").is_in(val_df["id"]))

#### ***Train dataset***

In [21]:
train_df.head(3)

id,origin_text,summary_text,lenght_origin_text,lenght_summary_text
i64,str,str,i64,i64
149068,"""By . Martha Kelner, Lara Gould…","""The two have an intense rivalr…",8993,315
68485,"""The boat that sank in the Medi…","""Overloaded ship capsized off t…",4867,335
275664,"""By . Lydia Warren For Mailonli…","""Kevin Ward Sr and his wife Pam…",5710,554


In [22]:
print('Number of rows:', train_df.shape[0])
print('Number of columns:', train_df.shape[1])

Number of rows: 253925
Number of columns: 5


#### ***Validation dataset***

In [23]:
val_df.head(3)

id,origin_text,summary_text,lenght_origin_text,lenght_summary_text
i64,str,str,i64,i64
39979,"""(CNN) -- If there was a predom…","""Cat Castellanos had been torn …",4261,309
147504,"""(CNN) -- President Obama is vi…","""President Obama's visit to Chi…",4655,338
104032,"""Beth Ditto was arrested in the…","""Beth Ditto was arrested in the…",3781,583


In [24]:
print('Number of rows:', val_df.shape[0])
print('Number of columns:', val_df.shape[1])

Number of rows: 31741
Number of columns: 5


#### ***Test dataset***

In [25]:
test_df.head(3)

id,origin_text,summary_text,lenght_origin_text,lenght_summary_text
i64,str,str,i64,i64
81120,"""By . Deni Kirkova . PUBLISHED:…","""The Londoner was in a car cras…",1632,204
269924,"""Finally, Sunderland have had a…","""Wes Brown looked to have earne…",7854,406
189690,"""By . Freya Noble . If the bigg…","""Drivers at alcohol delivery se…",2039,217


In [26]:
print('Number of rows:', test_df.shape[0])
print('Number of columns:', test_df.shape[1])

Number of rows: 31741
Number of columns: 5


## **Create Hugging Face dataset**

We will create a new empty dataset on Hugging Face.

### **Convert Polars DataFrames to Arrow tables**

In [27]:
train_arrow = train_df.to_arrow()
val_arrow = val_df.to_arrow()
test_arrow = test_df.to_arrow()

### **Save Arrow tables to local files**

In [28]:
save_dir = os.path.join('/kaggle','working')
train_arrow_path = os.path.join(save_dir,"train_dataset.parquet")
val_arrow_path = os.path.join(save_dir,"val_dataset.parquet")
test_arrow_path = os.path.join(save_dir,"test_dataset.parquet")

In [29]:
pq.write_table(train_arrow, train_arrow_path)
pq.write_table(val_arrow, val_arrow_path)
pq.write_table(test_arrow, test_arrow_path)

print(f"Saved Arrow files: {train_arrow_path}, {val_arrow_path}, {test_arrow_path}")

Saved Arrow files: /kaggle/working/train_dataset.parquet, /kaggle/working/val_dataset.parquet, /kaggle/working/test_dataset.parquet


### **Create a DatasetDict from Parquet files**

In [30]:
dataset_dict = DatasetDict({
    "train": Dataset.from_parquet(train_arrow_path),
    "validation": Dataset.from_parquet(val_arrow_path),
    "test": Dataset.from_parquet(test_arrow_path)
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

### **Define the repository name on Hugging Face**

In [31]:
repo_name = "KRadim/summary_dataset_en"

### **Upload the dataset to Hugging Face**

In [32]:
dataset_dict.push_to_hub(repo_name)
print(f"Dataset uploaded to Hugging Face at: https://huggingface.co/datasets/{repo_name}")

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/85 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   0%|          |  524kB /  222MB            

...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   1%|          |  525kB / 83.0MB            

README.md: 0.00B [00:00, ?B/s]

Dataset uploaded to Hugging Face at: https://huggingface.co/datasets/KRadim/summary_dataset_en
